Identify the class for each item, using the hint

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
seqdataIn = pd.read_json('/Volumes/GoogleDrive/My Drive/NYU/PhD/20 DL/project/data/seq-public.json', orient='records')
print(seqdataIn)

                  stem               options            category    id
0           7,9,-1,5,?        [4, 2, -1, -3]            sequence     0
1        3,2,5/3,3/2,?  [1/4, 7/5, 3/4, 2/5]            sequence     1
2           1,2,5,26,?    [34, 841, 677, 37]            sequence     2
3            2,12,30,?      [50, 65, 75, 56]            sequence     3
4        2,1,2/3,1/2,?  [3/4, 1/4, 2/5, 5/6]            sequence     4
...                ...                   ...                 ...   ...
1071  20 22 25 30 37 ?                    []  sequence-reasoning  1090
1072        0 1 3 10 ?                    []  sequence-reasoning  1091
1073       5 15 10 215                    []  sequence-reasoning  1092
1074        1 2 5 29 ?    [34, 841, 866, 37]  sequence-reasoning  1093
1075         2 12 30 ?      [50, 65, 75, 56]  sequence-reasoning  1094

[1076 rows x 4 columns]


In [3]:
seqDataAns =pd.read_json('/Volumes/GoogleDrive/My Drive/NYU/PhD/20 DL/project/data/seq-public.answer.json',orient='index')
print(seqDataAns)

     answer                                               hint
0       [4]      A<sub>n+1</sub>=2<sup>5-n</sup>-A<sub>n</sub>
1       [2]  3/1, 4/2, 5/3, 6/4\n\nA<sub>n+1</sub>=A<sub>n<...
2       [3]        A<sub>n+1</sub>=A<sub>n</sub><sup>2</sup>+1
3       [4]                           A<sub>n</sub>=2n\*(2n-1)
4       [3]             4/2, 4/4, 4/6, 4/8\n\nA<sub>n</sub>=2n
...     ...                                                ...
1090     48                A<sub>n+1</sub>-A<sub>n</sub>=P(n);
1091    102           A<sub>n\*2</sub>=A<sub>n\*2-1</sub>^2+2;
1092   -115   A<sub>n</sub>=A<sub>n-2</sub>^2-A<sub>n-1</sub>;
1093    [3]  A<sub>n</sub>=A<sub>n-1</sub>^2+A<sub>n-2</sub...
1094    [4]                      A<sub>n</sub>=(2\*n-1)\*2\*n;

[1076 rows x 2 columns]


In [4]:
def hint2txt(h):
    return h.replace('<sub>','_{').replace('</sub>','}').replace('<sup>','^{').replace('</sup>','}').replace('\n',' | ').replace('\*','*')
seqDataAns['hint'] = seqDataAns['hint'].map(lambda x: hint2txt(x))
print(seqDataAns['hint'])

0                                   A_{n+1}=2^{5-n}-A_{n}
1       3/1, 4/2, 5/3, 6/4 |  | A_{n+1}=A_{n}+1 |  | B...
2                                     A_{n+1}=A_{n}^{2}+1
3                                         A_{n}=2n*(2n-1)
4                        4/2, 4/4, 4/6, 4/8 |  | A_{n}=2n
                              ...                        
1090                                  A_{n+1}-A_{n}=P(n);
1091                               A_{n*2}=A_{n*2-1}^2+2;
1092                             A_{n}=A_{n-2}^2-A_{n-1};
1093                           A_{n}=A_{n-1}^2+A_{n-2}^2;
1094                                   A_{n}=(2*n-1)*2*n;
Name: hint, Length: 1076, dtype: object


In [26]:
seqDataAns['hint1'] = [item.split('=') for item in seqDataAns['hint']] #split the hint by '='

In [78]:
seqDataAns['classes'] = 0

In [79]:
seqDataAns

,answer,hint,hint1,classes
0,[4],A_{n+1}=2^{5-n}-A_{n},"[A_{n+1}, 2^{5-n}-A_{n}]",0
1,[2],"3/1, 4/2, 5/3, 6/4 | | A_{n+1}=A_{n}+1 | | B...","[3/1, 4/2, 5/3, 6/4 | | A_{n+1}, A_{n}+1 | |...",0
2,[3],A_{n+1}=A_{n}^{2}+1,"[A_{n+1}, A_{n}^{2}+1]",0
3,[4],A_{n}=2n*(2n-1),"[A_{n}, 2n*(2n-1)]",0
4,[3],"4/2, 4/4, 4/6, 4/8 | | A_{n}=2n","[4/2, 4/4, 4/6, 4/8 | | A_{n}, 2n]",0
...,...,...,...,...
1090,48,A_{n+1}-A_{n}=P(n);,"[A_{n+1}-A_{n}, P(n);]",0
1091,102,A_{n*2}=A_{n*2-1}^2+2;,"[A_{n*2}, A_{n*2-1}^2+2;]",0
1092,-115,A_{n}=A_{n-2}^2-A_{n-1};,"[A_{n}, A_{n-2}^2-A_{n-1};]",0
1093,[3],A_{n}=A_{n-1}^2+A_{n-2}^2;,"[A_{n}, A_{n-1}^2+A_{n-2}^2;]",0


In [129]:
for i in range(len(seqDataAns)):
    if i in [82,  257, 298, 306, 451, 452, 463, 464,468, 476,495, 498,499, 500,504,505,623,624,626,627,628]:
        continue
        
    if seqDataAns['classes'][i] == 0:
        y = False
        z = False
        if len(seqDataAns['hint1'][i])>1:
            y = re.search('A_{', seqDataAns['hint1'][i][1])
            z = re.search('(?<!_).*?n', seqDataAns['hint1'][i][1])

        if (y) and not(z):
            seqDataAns['classes'][i] = 1   #recursive A_{n}
            
        elif not(y) and (z):
            seqDataAns['classes'][i] = 2   #just the position n
        
        elif not(y) and not(z):
            seqDataAns['classes'][i] = 0   #none, or errors
            
        else:
            seqDataAns['classes'][i] = 3   #both
            
        

/Users/catalina/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/catalina/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [130]:
seqDataAns[seqDataAns['classes'] == 0]

,answer,hint,hint1,classes
86,"Y, Z",,[],0
91,[3],,[],0
97,[4],,[],0
110,[2],,[],0
118,[2],,[],0
...,...,...,...,...
1090,48,A_{n+1}-A_{n}=P(n);,"[A_{n+1}-A_{n}, P(n);]",0
1091,102,A_{n*2}=A_{n*2-1}^2+2;,"[A_{n*2}, A_{n*2-1}^2+2;]",0
1092,-115,A_{n}=A_{n-2}^2-A_{n-1};,"[A_{n}, A_{n-2}^2-A_{n-1};]",0
1093,[3],A_{n}=A_{n-1}^2+A_{n-2}^2;,"[A_{n}, A_{n-1}^2+A_{n-2}^2;]",0


In [138]:
seqDataAns.classes.value_counts() 

1    455
2    360
0    175
3     86
Name: classes, dtype: int64